# TensorFlowグラフの調査

https://www.tensorflow.org/tensorboard/graphs?hl=ja

## 概要

TensorBoardのグラフダッシュボードには、あなたのTensorFlowモデルを調べるための強力なツールです。モデルの構造の概念グラフをすばやく表示して、意図した設計と一致していることを確認できます。また、opレベルのグラフを表示して、TensorFlowがプログラムをどのように理解しているかを理解することもできます。運用レベルのグラフを調べると、モデルを変更する方法についての洞察が得られます。たとえば、トレーニングの進行が予想よりも遅い場合は、モデルを再設計できます。

このチュートリアルでは、グラフ診断データを生成し、TensorBoardのグラフダッシュボードで視覚化する方法の概要を説明します。 Fashion-MNISTデータセットの単純なKerasSequentialモデルを定義およびトレーニングし、モデルグラフをログに記録して調べる方法を学習します。また、新しい機能を使用して作成するためのグラフデータを生成するためにトレースAPIを使用しますtf.function注釈を。

## 設定

In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.7.0


In [3]:
import tensorboard
tensorboard.__version__

'2.7.0'

In [5]:
# Clear any logs from previous runs
%rm -rf ./logs/

## Kerasモデルを定義する

この例では、分類器は単純な4層のシーケンシャルモデルです。

In [6]:
# Define the model.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

2021-12-30 15:36:22.584587: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0

## モデルとログデータをトレーニングします

トレーニングの前に、定義Keras TensorBoardコールバックをログディレクトリを指定して、。このコールバックをModel.fit（）に渡すことにより、グラフデータがTensorBoardで視覚化するためにログに記録されるようにします。

In [8]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

Epoch 1/5
938/938 [==============================] - 2s 1ms/step - loss: 0.7029 - accuracy: 0.7587
Epoch 2/5
938/938 [==============================] - 1s 1ms/step - loss: 0.4901 - accuracy: 0.8284
Epoch 3/5
938/938 [==============================] - 1s 970us/step - loss: 0.4492 - accuracy: 0.8405
Epoch 4/5
938/938 [==============================] - 1s 945us/step - loss: 0.4260 - accuracy: 0.8481
Epoch 5/5
938/938 [==============================] - 1s 963us/step - loss: 0.4085 - accuracy: 0.8543


## Opレベルのグラフ

TensorBoardを起動し、UIが読み込まれるまで数秒待ちます。上部の[グラフ]をタップして、グラフダッシュボードを選択します。

In [9]:
# %tensorboard --logdir logs

オプションでTensorBoard.devを使用して、ホストされた共有可能な実験を作成することもできます。

In [10]:
!tensorboard dev upload \
  --logdir logs \
  --name "Sample op-level graph" \
  --one_shot

zsh:1: command not found: tensorboard


デフォルトでは、TensorBoardはOP-レベルのグラフを表示します。 （左側に、「デフォルト」タグが選択されているのがわかります。）グラフが反転していることに注意してください。データは下から上に流れるため、コードと比較して逆さまになっています。ただし、グラフがKerasモデルの定義と厳密に一致しており、他の計算ノードに余分なエッジがあることがわかります。

多くの場合、グラフは非常に大きいため、グラフの視覚化を操作できます。

* スクロールは、ズームイン、ズームアウトするには
* パンにドラッグ
* ダブルクリックする（ノードが他のノードのための容器であってもよい）、ノード拡張をトグル

ノードをクリックしてメタデータを表示することもできます。これにより、入力、出力、形状、およびその他の詳細を確認できます。

## 概念グラフ

実行グラフに加えて、TensorBoardも概念的なグラフを表示します。これは、Kerasモデルのみのビューです。これは、保存されたモデルを再利用していて、その構造を調べたり検証したりする場合に役立つことがあります。

概念グラフを表示するには、「keras」タグを選択します。この例では、崩壊したシーケンシャルノードが表示されます。ノードをダブルクリックして、モデルの構造を確認します。

## tf.functionsのグラフ

これまでの例では、Kerasモデルのグラフについて説明してきました。グラフは、Kerasレイヤーを定義し、Model.fit（）を呼び出すことによって作成されています。

あなたが使用する必要があるような状況が発生する可能性がありtf.functionに注釈を「サイン」の高性能TensorFlowグラフに、すなわち、変換、Pythonの演算機能を。このような状況のために、あなたはTensorBoardで可視化のためのサイン入りの機能をログに記録するTensorFlow概要トレースAPIを使用します。

サマリートレースAPIを使用するには：

* で関数を定義し、注釈を付けるtf.function
* 使用tf.summary.trace_on()すぐにあなたの関数の呼び出しサイトの前に。
* 渡すことによって、グラフにプロファイル情報（メモリ、CPU時間）を追加profiler=True
* 概要ファイルライター、呼び出しでtf.summary.trace_export()ログデータを保存します

次に、TensorBoardを使用して、関数がどのように動作するかを確認できます。

In [11]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


2021-12-30 16:31:56.831957: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2021-12-30 16:31:56.831982: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2021-12-30 16:31:56.942340: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2021-12-30 16:31:56.954635: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


これで、TensorBoardが理解できる関数の構造を確認できます。 [プロファイル]ラジオボタンをクリックして、CPUとメモリの統計を表示します。